Run this notebook to reduce data from the Nickel Telescope. Be sure to read the notes in between the cells.

In [ ]:
import numpy as np
import pandas as pd
import os
from astropy.io import fits

from overscan_subtraction import overscan_subtraction
from bias_subtraction import bias_subtraction
from dark_subtraction import dark_subtraction
from flat_division import flat_division
from correct_object_name import correct_object_name

The reduction requires the OBJECT names in the raw FITS files (which are set at the time of the observation) to be one of the following:
- "bias"
- "dark"
- "dome flat"
- "sky flat" (i.e., flats taken of the sky at sunset)
- "focus"
- your target names (can be anything)

If you need to correct OBJECT an in FITS headers of any files, do that below. If everything is correct, ignore the next cell.

In [ ]:
# correct_object(myfiles, "dark")

Now you're ready to reduce your data. First, get a list of all the raw data files. The name of the directory with the raw data should have format 'YYYY-MM-DD-nickel-raw'.

In [ ]:
rawdir = "" # path to directory with raw data
rawfiles = [rawdir + file for file in sorted(os.listdir(rawdir))]
# rawfiles

Create processing and reduced directories.

In [ ]:
rawdir_split = rawdir.split("/")
if rawdir_split[-1] == "":
    rootdir = "/".join(rawdir_split[:-2])+"/"
    datadir = rawdir_split[-2]+"/"
else:
    rootdir = "/".join(rawdir_split[:-1])+"/"
    datadir = rawdir_split[-1]+"/"
    
datadir_split = datadir.split("-")

procdir = "-".join(datadir_split[:4])+"-proc/"
os.makedirs(rootdir+procdir)
reddir = "-".join(datadir_split[:4])+"-red/"
os.makedirs(rootdir+reddir)

Do overscan subtraction.

In [ ]:
procdir = rootdir+procdir
procfiles = [procdir + file.split('.')[0] + '_proc.' + file.split('.')[1] for file in sorted(os.listdir(rawdir))]
overscan_subtraction(rawfiles, procfiles, 'yes')

Make a dataframe of all the files we want to continue reducing.

In [ ]:
obj_list = []
exptime_list = []
filt_list = []

for procfile in procfiles:
    hdul = fits.open(procfile)
    obj_list.append(hdul[0].header["OBJECT"])
    exptime_list.append(hdul[0].header["EXPTIME"])
    filt_list.append(hdul[0].header["FILTNAM"])
    hdul.close()
    
df_log = pd.DataFrame({
    "file": procfiles,
    "object": obj_list,
    "exptime": exptime_list,
    "filt": filt_list
    })

Do bias subtraction.

In [ ]:
# gather all the bias frames
biasfiles = list(df_log.file[df_log.object == 'bias'])

# average all of them into one
biasdata = []
for biasfile in biasfiles:
    hdul = fits.open(biasfile)
    biasdata.append(hdul[0].data)
    hdul.close()
bias = np.stack(biasdata).mean(axis=0)
# omit hot column so that it is properly flat-fielded out
bias[:,256] = 0

# gather all non-bias files
nonbiasfiles = list(df_log.file[df_log.object != 'bias'])

bias_subtraction(nonbiasfiles, nonbiasfiles, bias)

Do dark subtraction.

This can usually be skipped, since the Nickel CCD has a very low dark current, but it is included here for the sake of completeness.

In [ ]:
# if 'dark' in list(set(obj_list)):
#     darkexptimes = list(set(df_log.exptime[df_log.object == 'dark']))
#     for darkexptime in darkexptimes:
#         # find all files with this exposure time
#         darkfiles = list(df_log.file[(df_log.object == 'dark') & (df_log.exptime == darkexptime)])
#         flatfiles = list(df_log.file[((df_log.object == 'dome flat') | (df_log.object == 'sky flat')) &
#                                      (df_log.exptime == darkexptime)])
#         sciencefiles = list(df_log.file[(df_log.object != 'bias') &
#                                         (df_log.object != 'dark') &
#                                         (df_log.object != 'dome flat') &
#                                         (df_log.object != 'sky flat') &
#                                         (df_log.object != 'focus') &
#                                         (df_log.exptime == darkexptime)])
        
#         # calculate average dark frame
#         if len(darkfiles) > 1:
#             darkdata = []
#             for darkfile in darkfiles:
#                 hdul = fits.open(darkfile)
#                 darkdata.append(hdul[0].data)
#                 hdul.close()
#             dark = np.stack(darkdata).mean(axis=0)
#         else:
#             hdul = fits.open(darkfile)
#             dark = hdul[0].data
#             hdul.close()
        
#         # do dark subtraction
#         if len(flatfiles) > 0:
#             dark_subtraction(flatfiles, flatfiles, dark)
#         if len(sciencefiles) > 0:
#             dark_subtraction(sciencefiles, sciencefiles, dark)

# else:
#     print('No dark frames detected. Skipping dark subtraction.')

Do flat division.

Divide each pixel and then multiply all pixels by the average of the flat frame.

If sky (sunset) flats are available, those are used. If they are not available, dome flats are used.

In [ ]:
# use sky flats if available, use dome flats if not
if 'sky flat' in list(set(obj_list)):
    flattype = 'sky flat'
else:
    flattype = 'dome flat'
    
flatfilts = list(set(df_log.filt[df_log.object == flattype]))
for flatfilt in flatfilts:
    # find all the files with this filter
    flatfiles = list(df_log.file[(df_log.object == flattype) & (df_log.filt == flatfilt)])
    scienceobjects = list(set(df_log.object[(df_log.object != 'bias') &
                                            (df_log.object != 'dark') &
                                            (df_log.object != 'dome flat') &
                                            (df_log.object != 'sky flat') &
                                            (df_log.object != 'focus') &
                                            (df_log.filt == flatfilt)]))
    
    # calculate the average flat frame
    if len(flatfiles) > 1:
        flatdata = []
        for flatfile in flatfiles:
            hdul = fits.open(flatfile)
            flatdata.append(hdul[0].data)
            hdul.close()
        flat = np.stack(flatdata).mean(axis=0)
    else:
        hdul = fits.open(flatfile)
        flat = hdul[0].data
        hdul.close()
        
    if len(scienceobjects) > 0:
        for scienceobject in scienceobjects:
            sciencefiles = list(df_log.file[(df_log.object == scienceobject) &
                                            (df_log.filt == flatfilt)])
            
            # make a new directory for each science target / filter combination
            thisdir = scienceobject + '_' + flatfilt + '/'
            os.makedirs(rootdir+reddir+thisdir)

            # define reduced file names
            short_sciencefiles = [file.split('/')[-1] for file in sciencefiles]
            framenum = [frame.split('_')[0] for frame in short_sciencefiles]
            redfiles = [rootdir + reddir + thisdir + frame + '_red.fits' for frame in framenum]

            # do flat division
            if len(sciencefiles) > 0:
                flat_division(sciencefiles, redfiles, flat)

You're done! Your reduced images are now ready for your viewing.